In [ ]:
# DESAFIO II – CIÊNCIA E GOVERNANÇA DE DADOS
# Modelagem Preditiva e Explicabilidade do IDHM
# --------------------------------------------------
# Este notebook tem como objetivo:
# 1. Desenvolver modelos de Machine Learning para prever o IDHM
# 2. Comparar um modelo baseline com um modelo mais robusto
# 3. Interpretar os resultados utilizando SHAP values
# 4. Gerar insumos para dashboards interativos e recomendações estratégicas

# ==================================================
# 1. IMPORTAÇÃO DAS BIBLIOTECAS
# ==================================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import shap

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


# ==================================================
# 2. CARREGAMENTO DO DATASET
# ==================================================

DATA_PATH = "/home/alencaravelar/Desktop/zetta-lab/zetta-lab2/zetta-lab2/data/refined/base_udh_refined.csv"
OUTPUT_DIR = "/home/alencaravelar/Desktop/zetta-lab/zetta-lab2/zetta-lab2/outputs"

df = pd.read_csv(DATA_PATH)

print(df.head())
print(df.info())


# ==================================================
# 3. DEFINIÇÃO DE FEATURES E TARGET
# ==================================================
# X: Variáveis explicativas (indicadores socioeconômicos)
# y: Variável alvo (IDHM)

X = df.drop(columns=["IDHM"])
y = df["IDHM"]


# ==================================================
# 4. DIVISÃO EM TREINO E TESTE
# ==================================================
# Separação 80/20 para avaliação de generalização

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


# ==================================================
# 5. MODELO BASELINE – REGRESSÃO LINEAR
# ==================================================
# Objetivo: fornecer uma referência simples para justificar
# o uso de modelos não lineares

baseline = LinearRegression()
baseline_scores = cross_val_score(baseline, X_train, y_train, cv=5, scoring="r2")

print(f"Baseline Linear Regression R² médio (CV): {baseline_scores.mean():.3f}")


# ==================================================
# 6. MODELO PRINCIPAL: GRADIENT BOOSTING COM GRID SEARCH
# ==================================================
# Motivação:
# - No comparativo de modelos, Gradient Boosting obteve o melhor desempenho em teste
#   (R² Test ~ 0.99729) com baixo overfitting.
#
# Observação:
# - Gradient Boosting (GBDT) é sensível a hiperparâmetros; por isso aplicamos GridSearchCV.

gb = GradientBoostingRegressor(random_state=42)

param_grid_gb = {
    "n_estimators": [100, 200, 300],
    "learning_rate": [0.03, 0.05, 0.1],
    "max_depth": [2, 3, 5],
    "subsample": [0.8, 1.0],
    "min_samples_split": [2, 5],
    "min_samples_leaf": [1, 2],
}

grid_search_gb = GridSearchCV(
    estimator=gb,
    param_grid=param_grid_gb,
    cv=5,
    scoring="r2",
    n_jobs=-1,
)

grid_search_gb.fit(X_train, y_train)

best_gb = grid_search_gb.best_estimator_
print("Melhores hiperparâmetros (Gradient Boosting):", grid_search_gb.best_params_)


# ==================================================
# 7. AVALIAÇÃO DO MODELO (Gradient Boosting)
# ==================================================
# Avaliamos desempenho em treino e teste para verificar generalização e overfitting.

y_train_pred = best_gb.predict(X_train)
y_test_pred = best_gb.predict(X_test)

metrics = {
    "R2_Treino": r2_score(y_train, y_train_pred),
    "R2_Teste": r2_score(y_test, y_test_pred),
    "MAE": mean_absolute_error(y_test, y_test_pred),
    "RMSE": np.sqrt(mean_squared_error(y_test, y_test_pred)),
}

print(metrics)


# ==================================================
# 7.1 (Opcional) Comparativo com Random Forest (mantido como referência)
# ==================================================
# Você pediu para manter o RF como comparativo.
# Aqui usamos um RF simples (sem GridSearch) só para referência rápida.
# Se quiser, você pode manter o antigo bloco do RF com GridSearch em outra seção.

rf_ref = RandomForestRegressor(
    n_estimators=200, max_depth=20, random_state=42, max_features="sqrt"
)
rf_ref.fit(X_train, y_train)

rf_test_pred = rf_ref.predict(X_test)
rf_metrics = {
    "RF_R2_Teste": r2_score(y_test, rf_test_pred),
    "RF_MAE": mean_absolute_error(y_test, rf_test_pred),
    "RF_RMSE": np.sqrt(mean_squared_error(y_test, rf_test_pred)),
}
print("Métricas de referência (Random Forest):", rf_metrics)


# ==================================================
# 8. EXPLICABILIDADE COM SHAP VALUES (Gradient Boosting)
# ==================================================
# SHAP permite entender:
# - Quais variáveis mais impactam o IDHM
# - Direção do impacto (positivo ou negativo)
# - Explicações globais e locais
#
# Observação:
# - Para Gradient Boosting, TreeExplainer é apropriado (modelo baseado em árvores).

explainer = shap.TreeExplainer(best_gb)
shap_values = explainer.shap_values(X_test)


# ==================================================
# 9. VISUALIZAÇÃO GLOBAL – SUMMARY PLOT (sobrescreve arquivo)
# ==================================================

shap.summary_plot(shap_values, X_test, show=False)
plt.tight_layout()
plt.savefig(f"{OUTPUT_DIR}/shap_summary.jpg", dpi=300, bbox_inches="tight")
plt.close()


# ==================================================
# 10. IMPORTÂNCIA MÉDIA DAS VARIÁVEIS (SHAP) (sobrescreve CSV)
# ==================================================

shap_importance = np.abs(shap_values).mean(axis=0)

shap_df = (
    pd.DataFrame({"Feature": X.columns, "Mean_SHAP_Value": shap_importance})
    .sort_values(by="Mean_SHAP_Value", ascending=False)
)

shap_df.to_csv(f"{OUTPUT_DIR}/shap_importance_results.csv", index=False)

print(shap_df.head(10))


# ==================================================
# 11. INTERPRETAÇÃO LOCAL (EXEMPLO) (sobrescreve arquivo)
# ==================================================

idx = 0  # índice de exemplo

exp = shap.Explanation(
    values=shap_values[idx],
    base_values=explainer.expected_value,
    data=X_test.iloc[idx],
    feature_names=X.columns,
)

shap.plots.waterfall(exp, show=False)
plt.tight_layout()
plt.savefig(f"{OUTPUT_DIR}/shap_local_explanation.jpg", dpi=300, bbox_inches="tight")
plt.close()


# ==================================================
# FIM DO NOTEBOOK
# ==================================================


   T_ANALF15M  T_ATRASO_2_BASICO  T_FUND18M  AGUA_ESGOTO  T_DENS  T_LIXO  \
0       13.65              13.19      27.66         2.83   35.62   75.28   
1        5.78              11.53      43.00         0.26   36.25   99.87   
2        9.56              19.76      33.16         1.26   50.46   93.86   
3       13.71              26.05      23.03         6.06   44.57   85.60   
4       12.18              26.83      28.00         0.40   54.62   99.93   

   GINI   PPOB  T_FUNDIN18MINF  P_FORMAL  T_DES18M  RAZDEP   IDHM  
0  0.52  61.05           62.61     34.70     11.92   63.76  0.592  
1  0.40  31.70           39.84     64.73     15.43   44.37  0.664  
2  0.44  49.50           46.76     62.37     19.76   65.93  0.582  
3  0.41  62.13           53.18     62.97     25.15   60.15  0.529  
4  0.40  67.85           50.09     61.20     21.91   47.33  0.531  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1228 entries, 0 to 1227
Data columns (total 13 columns):
 #   Column             Non-

/home/alencaravelar/anaconda3/lib/python3.13/site-packages/numpy/ma/core.py:2881: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Melhores hiperparâmetros: {'max_depth': 20, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
{'R2_Treino': 0.9995695654420702, 'R2_Teste': 0.9972775302642526, 'MAE': np.float64(0.003376077235772435), 'RMSE': np.float64(0.005669132980368132)}
             Feature  Mean_SHAP_Value
2          T_FUND18M         0.026032
7               PPOB         0.018661
8     T_FUNDIN18MINF         0.018213
4             T_DENS         0.011604
1  T_ATRASO_2_BASICO         0.006096


Exception ignored in: <function ResourceTracker.__del__ at 0x779748587ba0>
Traceback (most recent call last):
  File "/home/alencaravelar/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 82, in __del__
  File "/home/alencaravelar/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 91, in _stop
  File "/home/alencaravelar/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 116, in _stop_locked
ChildProcessError: [Errno 10] No child processes
Exception ignored in: <function ResourceTracker.__del__ at 0x7fb151787ba0>
Traceback (most recent call last):
  File "/home/alencaravelar/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 82, in __del__
  File "/home/alencaravelar/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 91, in _stop
  File "/home/alencaravelar/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 116, in _stop_locked
ChildProcessError: [Errno 10] No child processes
Exceptio